# Pose Correction Workout System

This notebook implements a system to help users correct their exercise form using a webcam and MediaPipe pose detection. The system:
- Records reference animations for three exercises (dumbbell curls, exercise2, exercise3) with a 5-second countdown.
- Displays an animated blue stick figure of the correct form.
- Overlays the user's live pose as a stick figure (red when incorrect, green when correct within a 10-degree angle leeway).
- Shows only stick figures on a black background, not the camera feed.
- Uses relative angles for form correction, so position (e.g., center vs. side) doesn't affect correctness.

**Note**: Replace 'exercise2' and 'exercise3' with the actual exercise names from your images. For best results, run this code as a `.py` script outside Jupyter, as `cv2.imshow` may not work reliably in Jupyter. Alternatively, use the commented matplotlib code for Jupyter display (slower).

## Imports and Setup

In [1]:
import cv2
import mediapipe as mp
import time
import pickle
import numpy as np
# For Jupyter display (optional, slower)
# from matplotlib import pyplot as plt
# %matplotlib inline

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

## Function to Calculate Angle

Helper function to compute the angle between three landmarks (e.g., shoulder-elbow-wrist) for form comparison.

In [2]:
def calculate_angle(a, b, c):
    # a, b, c are (x, y) tuples
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ab = a - b
    bc = c - b
    cosine_angle = np.dot(ab, bc) / (np.linalg.norm(ab) * np.linalg.norm(bc) + 1e-6)
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

## Function to Record Reference Animation

Records a sequence of pose landmarks after a 5-second countdown. Saves to a pickle file.

In [3]:
def record_reference(exercise_name):
    pose = mp_pose.Pose()
    cap = cv2.VideoCapture(0)
    references = []
    
    print(f"Preparing to record {exercise_name}. Get ready!")
    
    # 5-second countdown
    start_time = time.time()
    while time.time() - start_time < 5:
        ret, frame = cap.read()
        if not ret:
            break
        countdown = 5 - int(time.time() - start_time)
        cv2.putText(frame, f'Starting in {countdown}...', (50, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.imshow('Recording Reference', frame)
        # For Jupyter (uncomment to use, slower)
        # plt.clf()
        # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        # plt.axis('off')
        # plt.show()
        # plt.pause(0.001)
        cv2.waitKey(1)
    
    print(f"Recording for {exercise_name}. Perform the exercise correctly. Press 'q' to stop.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        
        if results.pose_landmarks:
            lm_list = []
            for lm in results.pose_landmarks.landmark:
                lm_list.append((lm.x, lm.y, lm.z, lm.visibility))
            references.append(lm_list)
        
        cv2.imshow('Recording Reference', frame)
        # For Jupyter (uncomment to use, slower)
        # plt.clf()
        # plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        # plt.axis('off')
        # plt.show()
        # plt.pause(0.001)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    with open(f'{exercise_name}.pkl', 'wb') as f:
        pickle.dump(references, f)
    
    print(f"Recorded {len(references)} frames for {exercise_name}.")

## Record Reference Animations

Run each cell below to record the reference animation for each exercise. A 5-second countdown will appear. Perform the exercise correctly and slowly. Press 'q' to stop recording.

### Record Dumbbell Curls

In [4]:
record_reference('dumbbell_curls')

Preparing to record dumbbell_curls. Get ready!
Recording for dumbbell_curls. Perform the exercise correctly. Press 'q' to stop.
Recorded 162 frames for dumbbell_curls.


### Record Exercise 2

Replace 'exercise2' with the actual exercise name from your first image.

In [5]:
record_reference('exercise2')

Preparing to record exercise2. Get ready!
Recording for exercise2. Perform the exercise correctly. Press 'q' to stop.
Recorded 107 frames for exercise2.


### Record Exercise 3

Replace 'exercise3' with the actual exercise name from your second image.

In [6]:
record_reference('exercise3')

Preparing to record exercise3. Get ready!
Recording for exercise3. Perform the exercise correctly. Press 'q' to stop.
Recorded 118 frames for exercise3.


## Function to Perform the Workout

Loads the reference animation and displays it as a blue stick figure. The user's live pose is overlaid as a stick figure (red if incorrect, green if correct within a 10-degree angle leeway). Only stick figures are shown on a black background. The reference animates at ~10 FPS. Press 'q' to quit.

In [7]:
def perform_exercise(exercise_name):
    with open(f'{exercise_name}.pkl', 'rb') as f:
        reference_sequence = pickle.load(f)
    
    if not reference_sequence:
        print("No reference data found.")
        return
    
    pose = mp_pose.Pose()
    cap = cv2.VideoCapture(0)
    ref_index = 0
    animation_fps = 10  # Speed of reference animation
    frame_delay = 1.0 / animation_fps
    last_ref_time = time.time()
    angle_threshold = 10  # Leeway for angle correctness (degrees)
    
    pose_connections = mp_pose.POSE_CONNECTIONS
    
    print(f"Starting {exercise_name}. Match the blue stick figure. Press 'q' to quit.")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        height, width, _ = frame.shape
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb)
        
        # Create black background
        black = np.zeros((height, width, 3), dtype=np.uint8)
        
        # Draw reference stick figure in blue (lines only)
        ref_lm_list = reference_sequence[ref_index]
        for conn in pose_connections:
            start = ref_lm_list[conn[0]]
            end = ref_lm_list[conn[1]]
            if start[3] > 0.1 and end[3] > 0.1:  # Check visibility
                cv2.line(black,
                         (int(start[0] * width), int(start[1] * height)),
                         (int(end[0] * width), int(end[1] * height)),
                         (255, 0, 0),  # Blue in BGR
                         2)
        
        # Draw user's stick figure with per-segment coloring
        if results.pose_landmarks:
            user_lm_list = []
            for lm in results.pose_landmarks.landmark:
                user_lm_list.append((lm.x, lm.y, lm.z, lm.visibility))
            
            for conn in pose_connections:
                idx1, idx2 = conn
                ref_start = ref_lm_list[idx1]
                ref_end = ref_lm_list[idx2]
                user_start = user_lm_list[idx1]
                user_end = user_lm_list[idx2]
                
                if user_start[3] > 0.1 and user_end[3] > 0.1 and ref_start[3] > 0.1 and ref_end[3] > 0.1:
                    # Find a third point for angle calculation (e.g., parent joint)
                    # Use a simple heuristic: pick a connected landmark
                    parent_idx = None
                    for c in pose_connections:
                        if c[0] == idx2 and c[1] != idx1:
                            parent_idx = c[1]
                            break
                        elif c[1] == idx2 and c[0] != idx1:
                            parent_idx = c[0]
                            break
                    
                    if parent_idx is not None and ref_lm_list[parent_idx][3] > 0.1 and user_lm_list[parent_idx][3] > 0.1:
                        ref_parent = ref_lm_list[parent_idx]
                        user_parent = user_lm_list[parent_idx]
                        
                        # Calculate angles
                        ref_angle = calculate_angle((ref_start[0], ref_start[1]),
                                                  (ref_end[0], ref_end[1]),
                                                  (ref_parent[0], ref_parent[1]))
                        user_angle = calculate_angle((user_start[0], user_start[1]),
                                                    (user_end[0], user_end[1]),
                                                    (user_parent[0], user_parent[1]))
                        
                        # Check if angles are within leeway
                        color = (0, 255, 0) if abs(ref_angle - user_angle) < angle_threshold else (0, 0, 255)  # Green or red
                        
                        cv2.line(black,
                                 (int(user_start[0] * width), int(user_start[1] * height)),
                                 (int(user_end[0] * width), int(user_end[1] * height)),
                                 color,
                                 2)
        
        # Display using OpenCV (preferred)
        cv2.imshow('Workout Form Correction', black)
        
        # For Jupyter (uncomment to use, slower)
        # plt.clf()
        # plt.imshow(black)
        # plt.axis('off')
        # plt.show()
        # plt.pause(0.001)
        
        # Advance reference animation
        current_time = time.time()
        if current_time - last_ref_time >= frame_delay:
            last_ref_time = current_time
            ref_index = (ref_index + 1) % len(reference_sequence)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

## Perform the Exercises

Run each cell to start the workout session for each exercise. Match the blue stick figure's movements.

### Perform Dumbbell Curls

In [8]:
perform_exercise('dumbbell_curls')

Starting dumbbell_curls. Match the blue stick figure. Press 'q' to quit.


In [9]:
stop

NameError: name 'stop' is not defined

### Perform Exercise 2

In [10]:
perform_exercise('exercise2')

Starting exercise2. Match the blue stick figure. Press 'q' to quit.


### Perform Exercise 3

In [11]:
perform_exercise('exercise3')

Starting exercise3. Match the blue stick figure. Press 'q' to quit.
